In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
import plotly.express as px

In [50]:
session_data = pd.read_csv("/Users/florianprivat/PycharmProjects/djdata/session_data/spotify_session_2024-08-02_11:28:39.csv")
session_data = session_data.drop_duplicates(subset='name')
session_data = session_data.reset_index(drop=True)
profile = ProfileReport(session_data, title="German Open Set 2024")

In [27]:
profile.to_file("german_open_dj_session.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
session_data['tempo'] = [tempo/2 if tempo > 120 else tempo for tempo in session_data['tempo']]

In [54]:
# Create the line graph using Plotly Express
fig = px.line(session_data, x="name", y="tempo")

# Set the title and axis labels
fig.update_layout(
    title="Song Speed Over Time",
    xaxis_title="Song",
    yaxis_title="Tempo (bpm)"
)

# Display the graph
# all three of these worked
fig.show(renderer='iframe_connected')

In [59]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import make_interp_spline

# Convert session_data to a format suitable for Plotly
name = session_data['name']
tempo = session_data['tempo']

# Spline interpolation for smoothing
x = np.arange(len(name))
x_smooth = np.linspace(x.min(), x.max(), 300)
spl = make_interp_spline(x, tempo, k=3)  # k=3 for cubic spline
tempo_smooth = spl(x_smooth)

# Create the smooth line graph using Plotly Graph Objects
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_smooth,
    y=tempo_smooth,
    mode='lines',
    name='Smoothed Tempo',
    line=dict(color='royalblue', width=2)
))

# Set the x-axis tick values to correspond to song names
fig.update_xaxes(
    tickvals=x,
    ticktext=name
)

# Set the title and axis labels
fig.update_layout(
    title="Smoothed Song Speeds in Order",
    xaxis_title="Song",
    yaxis_title="Tempo (bpm)",
    template='plotly_white',
    width=800,
    height=600,
    margin=dict(l=50, r=50, t=80, b=50),
    showlegend=False
)

# Display the graph
fig.show(renderer='iframe_connected')


In [58]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import make_interp_spline

# Convert session_data to a format suitable for Plotly
name = session_data['name']
tempo = session_data['tempo']

# Spline interpolation for smoothing
x = np.arange(len(name))
x_smooth = np.linspace(x.min(), x.max(), 300)
spl = make_interp_spline(x, tempo, k=3)  # k=3 for cubic spline
tempo_smooth = spl(x_smooth)

# Create the smooth line graph using Plotly Graph Objects
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_smooth,
    y=tempo_smooth,
    mode='lines',
    name='Smoothed Tempo',
    line=dict(color='royalblue', width=2)
))

# Set the x-axis tick values to correspond to song names, displaying only every second song
tickvals = x[::2]
ticktext = name[::2]

fig.update_xaxes(
    tickvals=tickvals,
    ticktext=ticktext
)

# Set the title and axis labels
fig.update_layout(
    title="Song Speeds in Order",
    xaxis_title="Song",
    yaxis_title="Tempo (bpm)",
    template='plotly_white',
    width=800,
    height=600,
    margin=dict(l=50, r=50, t=80, b=50),
    showlegend=False
)

# Display the graph
fig.show(renderer='iframe_connected')


In [30]:
session_data.columns

Index(['Unnamed: 0', 'artists', 'available_markets', 'disc_number',
       'duration_ms_x', 'explicit', 'href', 'id_x', 'is_local', 'name',
       'popularity', 'preview_url', 'track_number', 'type_x', 'uri_x',
       'album.album_type', 'album.artists', 'album.available_markets',
       'album.external_urls.spotify', 'album.href', 'album.id', 'album.images',
       'album.name', 'album.release_date', 'album.release_date_precision',
       'album.total_tracks', 'album.type', 'album.uri', 'external_ids.isrc',
       'external_urls.spotify', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type_y', 'id_y', 'uri_y', 'track_href',
       'analysis_url', 'duration_ms_y', 'time_signature'],
      dtype='object')

In [31]:
session_data[['name', 'tempo']].head()

,name,tempo
0,One Thing Right,88.042
1,J'me tire,95.017
2,Trapped in a Dream,89.048
3,The Way I Want It,97.981
4,Fabulous,109.014


In [42]:
session_data[['acousticness']].describe()

,acousticness
count,40.000000
mean,0.325906
std,0.256080
min,0.008920
25%,0.094275
50%,0.270000
75%,0.429500
max,0.917000


In [60]:
song_dimensions = ['danceability','speechiness','acousticness','instrumentalness','liveness','valence','energy']

In [61]:
spider_data = session_data[song_dimensions]
for col in song_dimensions:
    spider_data[col] = (spider_data[col] - spider_data[col].min()) / (spider_data[col].max() - spider_data[col].min())

/var/folders/z7/9_928_ds1l9_46_hmcd5w5tc0000gp/T/ipykernel_91593/695030624.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
import plotly.graph_objects as go

# Define the categories and the values for each category
categories = ['danceability','acousticness','speechiness','instrumentalness','liveness','valence']
values = [spider_data[cat].mean() for cat in categories]

# Append the first value to the end to close the loop of the spider web
values += values[:1]

# Create the radar chart
fig = go.Figure(
    data=[
        go.Scatterpolar(
            r=values,
            theta=categories + [categories[0]],
            fill='toself'
        )
    ]
)

# Customize the layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, max(values)+0.1*max(values)]
        )
    ),
    showlegend=False
)

# Show the plot
fig.show(renderer='iframe_connected')

In [63]:
session_data[session_data['liveness']>0.5]

,Unnamed: 0,artists,available_markets,disc_number,duration_ms_x,explicit,href,id_x,is_local,name,...,liveness,valence,tempo,type_y,id_y,uri_y,track_href,analysis_url,duration_ms_y,time_signature
0,0,[{'external_urls': {'spotify': 'https://open.s...,[],1,181823,False,https://api.spotify.com/v1/tracks/4hPpVbbakQNv...,4hPpVbbakQNv8YTHYaOJP4,False,One Thing Right,...,0.582,0.442,88.042,audio_features,4hPpVbbakQNv8YTHYaOJP4,spotify:track:4hPpVbbakQNv8YTHYaOJP4,https://api.spotify.com/v1/tracks/4hPpVbbakQNv...,https://api.spotify.com/v1/audio-analysis/4hPp...,181824,4
